In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import img_to_array


In [2]:
accessories=r"C:\Users\yousif\OneDrive\Desktop\slash dataset\accessories"
fashion=r"C:\Users\yousif\OneDrive\Desktop\slash dataset\fashion"

In [3]:
accessories_output=r"C:\Users\yousif\OneDrive\Desktop\output\accessories"
fashion_output=r"C:\Users\yousif\OneDrive\Desktop\output\fashion"

In [4]:
crop_width0, crop_height0, crop_width1, crop_height1 = 0, 460, 550, 950

In [5]:
def crop_data(output_path,dataset_path,crop_width0, crop_height0, crop_width1, crop_height1):
    for filename in os.listdir(dataset_path):
        image_path=os.path.join(dataset_path,filename)
        img=Image.open(image_path)
        cropped_img=img.crop((crop_width0, crop_height0, crop_width1, crop_height1))
        output_image_path=os.path.join(output_path,filename)
        cropped_img.save(output_image_path)

In [6]:
crop_data(accessories_output,accessories,crop_width0, crop_height0, crop_width1, crop_height1)

In [7]:
crop_data(fashion_output,fashion,crop_width0, crop_height0, crop_width1, crop_height1)

In [8]:
dataset_path=r"C:\Users\yousif\OneDrive\Desktop\output"

categories=["accessories","fashion"]


In [9]:
data_splits = {'train': {}, 'validation': {}, 'test': {}}

In [10]:
for category in categories:
    category_path=os.path.join(dataset_path,category)
    images=os.listdir(category_path)
    
    train_images,temp_images=train_test_split(images,test_size=0.2,random_state=42)
    val_images,test_images=train_test_split(temp_images,test_size=0.5,random_state=42)
    
    data_splits['train'][category] = train_images
    data_splits['validation'][category] = val_images
    data_splits['test'][category] = test_images


In [11]:
for split, categories_dict in data_splits.items():
    print(f"{split.capitalize()} Split:")
    for category, images in categories_dict.items():
        print(f"{category}: {len(images)} images")
    print()

Train Split:
accessories: 86 images
fashion: 83 images

Validation Split:
accessories: 11 images
fashion: 10 images

Test Split:
accessories: 11 images
fashion: 11 images



In [12]:
train_dir      =  r"C:\Users\yousif\OneDrive\Desktop\dataset_modified\train"
validation_dir = r"C:\Users\yousif\OneDrive\Desktop\dataset_modified\val"
test_dir       = r"C:\Users\yousif\OneDrive\Desktop\dataset_modified\test"



for split, categories_dict in data_splits.items():
    for category, images in categories_dict.items():
        category_dir=os.path.join(eval(f"{split}_dir"),category)
        os.makedirs(category_dir,exist_ok=True)
        
        for image in images:
            src=os.path.join(dataset_path,category,image)
            dst=os.path.join(category_dir,image)
            shutil.move(src,dst)

In [13]:
dataset_dir=r"C:\Users\yousif\OneDrive\Desktop\dataset_modified"


In [30]:
filepaths = []
categories = []
splits = []

for split in ["train", "val", "test"]:
    split_dir = os.path.join(dataset_dir, split)
    
    for category in os.listdir(split_dir):
        category_dir = os.path.join(split_dir, category)
        
        # This loop should be nested within the loop that iterates over categories
        for filename in os.listdir(category_dir):
            filepath = os.path.join(category_dir, filename)
            filepaths.append(filepath)
            categories.append(category)
            splits.append(split)

data = {
    'Filepath': filepaths,
    'Category': categories,
    'Split': splits
}
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


                                              Filepath     Category  Split
0    C:\Users\yousif\OneDrive\Desktop\dataset_modif...  accessories  train
1    C:\Users\yousif\OneDrive\Desktop\dataset_modif...  accessories  train
2    C:\Users\yousif\OneDrive\Desktop\dataset_modif...  accessories  train
3    C:\Users\yousif\OneDrive\Desktop\dataset_modif...  accessories  train
4    C:\Users\yousif\OneDrive\Desktop\dataset_modif...  accessories  train
..                                                 ...          ...    ...
212  C:\Users\yousif\OneDrive\Desktop\dataset_modif...      fashion   test
213  C:\Users\yousif\OneDrive\Desktop\dataset_modif...      fashion   test
214  C:\Users\yousif\OneDrive\Desktop\dataset_modif...      fashion   test
215  C:\Users\yousif\OneDrive\Desktop\dataset_modif...      fashion   test
216  C:\Users\yousif\OneDrive\Desktop\dataset_modif...      fashion   test

[217 rows x 3 columns]


In [32]:
# Filter the DataFrame to show only entries where the category is "accessory"
accessory_df = df[df['Category'] == 'fashion']

# Display the filtered DataFrame
print(accessory_df)


                                              Filepath Category  Split
88   C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion  train
89   C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion  train
90   C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion  train
91   C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion  train
92   C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion  train
..                                                 ...      ...    ...
212  C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion   test
213  C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion   test
214  C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion   test
215  C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion   test
216  C:\Users\yousif\OneDrive\Desktop\dataset_modif...  fashion   test

[107 rows x 3 columns]


In [47]:
train_df=df[df["Split"]=="train"]
test_df=df[df["Split"]=="test"]
val_df=df[df["Split"]=="val"]


In [53]:
def create_image_generators(train_df,test_df,val_df):
    train_generator=tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,
                                                                    rescale=1./255,
                                                                    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
    
    test_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
    
    val_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                 preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
    
    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Category',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=True,
        seed=42,
    )
    

    val_images = val_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Category',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=True,
        seed=42
    )
        
        
        
        
        
        
    test_images = test_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Category',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=True,
        seed=42
    )
    return train_images, val_images, test_images

In [50]:
def build_model():
    # Build and return the ResNet50-based model
    pretrained_model = tf.keras.applications.resnet50.ResNet50(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg'
    )

    pretrained_model.trainable = False

    inputs = pretrained_model.input
    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(50, activation='relu')(x)
    outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

In [60]:
def train_and_evaluate_slash_model(train_images, val_images, test_images, save_path="slash_model.h5"):
    model = build_model()
    print("-------Training Slash Model-------")

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = model.fit(train_images, validation_data=val_images, epochs=25, callbacks=[callbacks])

    # Evaluate the model on the test set
    print("-------Evaluating Slash Model-------")
    test_loss, test_accuracy = model.evaluate(test_images, verbose=1)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

    # Save the model
    print("-------Saving Model-------")
    model.save(save_path)

    return model

In [61]:
def main():
    # Create image generators
    train_images, val_images, test_images = create_image_generators(train_df, val_df, test_df)

    # Train and evaluate the Slash model
    model = train_and_evaluate_slash_model(train_images, val_images, test_images)

if __name__ == "__main__":
    main()


Found 171 validated image filenames belonging to 2 classes.
Found 171 validated image filenames belonging to 2 classes.
Found 171 validated image filenames belonging to 2 classes.


C:\Users\yousif\anaconda3\lib\site-packages\keras\preprocessing\image.py:1137: UserWarning: Found 3 invalid image filename(s) in x_col="Filepath". These filename(s) will be ignored.
  warnings.warn(
C:\Users\yousif\anaconda3\lib\site-packages\keras\preprocessing\image.py:1137: UserWarning: Found 3 invalid image filename(s) in x_col="Filepath". These filename(s) will be ignored.
  warnings.warn(
C:\Users\yousif\anaconda3\lib\site-packages\keras\preprocessing\image.py:1137: UserWarning: Found 3 invalid image filename(s) in x_col="Filepath". These filename(s) will be ignored.
  warnings.warn(


-------Training Slash Model-------
Epoch 1/25
3/3 [==============================] - 24s 9s/step - loss: 0.7146 - accuracy: 0.4094 - val_loss: 0.7022 - val_accuracy: 0.4971
Epoch 2/25
3/3 [==============================] - 19s 8s/step - loss: 0.7104 - accuracy: 0.4854 - val_loss: 0.7015 - val_accuracy: 0.4971
Epoch 3/25
3/3 [==============================] - 18s 7s/step - loss: 0.6985 - accuracy: 0.4971 - val_loss: 0.6872 - val_accuracy: 0.5029
Epoch 4/25
3/3 [==============================] - 19s 8s/step - loss: 0.6839 - accuracy: 0.5322 - val_loss: 0.6782 - val_accuracy: 0.7076
Epoch 5/25
3/3 [==============================] - 19s 8s/step - loss: 0.6812 - accuracy: 0.6491 - val_loss: 0.6748 - val_accuracy: 0.6257
Epoch 6/25
3/3 [==============================] - 18s 7s/step - loss: 0.6744 - accuracy: 0.6257 - val_loss: 0.6687 - val_accuracy: 0.6374
Epoch 7/25
3/3 [==============================] - 18s 7s/step - loss: 0.6666 - accuracy: 0.6667 - val_loss: 0.6611 - val_accuracy: 0.7661